Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB3

Wrote a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, November 26 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, November 27
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [32]:
from random import choices, randint, random

import lab3_lib

In [33]:
fitness = lab3_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

01101010100110011111000110010000111100110111000111: 15.33%
00001000001011011010001100001000101000100100100011: 7.34%
01111011101110100000100000010110000001010001111110: 7.34%
00000100111001011110110101010111110010000010000011: 29.56%
11000100000111111100110011101010111101100110111101: 9.11%
11111010011010010111111000101101010001010100111100: 15.33%
10110011001000000010110001100010001001110011111010: 7.34%
10000000001000110110110001101111001010001100000100: 9.36%
10111101001101100000001100101010110011010111111101: 9.13%
10111100101011111010101000101010111110101100110010: 19.11%
10


This code implements a Genetic Algorithm with Simulated Annealing-based Local Search to solve an optimization problem. The `LocalSearch` class represents local search based on simulated annealing. It explores neighbors of a given solution and accepts or rejects them based on fitness and a parameter similar to temperature.

The algorithm initializes a population of binary individuals and iteratively optimizes them using local search. Crossover and mutation operations are applied, and the process continues until convergence or a maximum number of generations.

In the population mutation, for each individual and each gene in the genome, there is a probability (`self.mutation_rate`) that the gene is inverted (from 0 to 1 or from 1 to 0).
The crossover method creates new individuals through two-point crossover. It randomly selects two parents from the population, randomly selects two crossover points, and creates two children by combining the parents' genes between these points.
In summary, mutation randomly changes genes of individual individuals, while crossover combines genes from two parents to create new individuals. Both of these processes contribute to diversifying the population and exploring new solutions in the search space.

In [34]:
from random import choices, randint, random, uniform
from math import exp

class GeneticAlgorithm:
    def __init__(self, problem, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5):
        """
        Initialize the Genetic Algorithm parameters.

        Args:
        - problem: The optimization problem to solve.
        - genome_size: The size of the binary genome for each individual.
        - population_size: The number of individuals in the population.
        - generations: The maximum number of generations to run the algorithm.
        - mutation_rate: The probability of mutation for each bit in the genome.
        - local_search_iterations: The number of iterations for the local search algorithm.
        - convergence_threshold: The threshold for consecutive generations without improvement to terminate the algorithm.
        """
        self.problem = problem
        self.genome_size = genome_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold

    def run(self):
        """
        Run the Genetic Algorithm.

        Returns:
        - best_individual_after_local: The best individual after local search.
        """
        # Initialize the population
        population = [choices([0, 1], k=self.genome_size) for _ in range(self.population_size)]

        best_individual_after_local = None
        consecutive_no_improvement = 0

        for generation in range(self.generations):
            # Evaluate the fitness of the population
            fitness_scores = [self.problem(individual) for individual in population]
            best_individual = population[fitness_scores.index(max(fitness_scores))]

            # Apply local search to the best individual
            local_search = LocalSearch(self.problem, best_individual, iterations=self.local_search_iterations)
            local_search.run()
            best_individual_after_local = local_search.best_individual
            fitness_after_local = local_search.best_fitness

            # If fitness after local search improves, reset the no improvement counter
            if fitness_after_local > self.problem(best_individual):
                consecutive_no_improvement = 0
            else:
                consecutive_no_improvement += 1

            # If no improvement for a certain number of consecutive generations, terminate the algorithm
            if consecutive_no_improvement >= self.convergence_threshold:
                break

            # If fitness after local search improves, replace an individual in the population
            if fitness_after_local > self.problem(best_individual):
                index_to_replace = fitness_scores.index(min(fitness_scores))
                population[index_to_replace] = best_individual_after_local

            # Update the population using crossover and mutation
            new_population = self.crossover(population)
            self.mutate(new_population)
            population = new_population

        # Return the best individual after all generations
        return best_individual_after_local

    def crossover(self, population):
        """
        Apply crossover to generate new individuals from the existing population.

        Args:
        - population: The current population.

        Returns:
        - new_population: The new population after crossover.
        """
        new_population = []

        for _ in range(self.population_size // 2):
            # Tournament selection of parents
            parent1 = choices(population)[0]
            parent2 = choices(population)[0]

            # Two-point crossover
            crossover_points = sorted([randint(0, self.genome_size - 1) for _ in range(2)])

            child1 = parent1[:crossover_points[0]] + parent2[crossover_points[0]:crossover_points[1]] + parent1[crossover_points[1]:]
            child2 = parent2[:crossover_points[0]] + parent1[crossover_points[0]:crossover_points[1]] + parent2[crossover_points[1]:]

            new_population.extend([child1, child2])

        return new_population

    def mutate(self, population):
        """
        Apply mutation to each individual in the population.

        Args:
        - population: The current population.
        """
        for i in range(len(population)):
            for j in range(self.genome_size):
                if random() < self.mutation_rate:
                    population[i][j] = 1 - population[i][j]


class LocalSearch:
    def __init__(self, problem, initial_solution, iterations=10):
        """
        Initialize the Local Search algorithm parameters.

        Args:
        - problem: The optimization problem to solve.
        - initial_solution: The initial solution for local search.
        - iterations: The number of iterations for the local search.
        """
        self.problem = problem
        self.genome_size = len(initial_solution)
        self.best_individual = initial_solution
        self.best_fitness = problem(self.best_individual)
        self.iterations_without_improvement = 0
        self.iterations = iterations

    def run(self):
        """Run the Local Search algorithm."""
        for _ in range(self.iterations):
            neighbor = self.generate_neighbor(self.best_individual)
            neighbor_fitness = self.problem(neighbor)

            if neighbor_fitness > self.best_fitness:
                self.best_individual = neighbor
                self.best_fitness = neighbor_fitness
                self.iterations_without_improvement = 0
            else:
                self.iterations_without_improvement += 1

            if self.best_fitness == 1.0:
                break  # Exit if maximum fitness is reached

    def generate_neighbor(self, individual):
        """
        Generate a neighbor using Simulated Annealing.

        Args:
        - individual: The current individual.

        Returns:
        - neighbor: The generated neighbor.
        """
        # Simulated Annealing: Generate a neighbor by flipping bits with decreasing probability
        index_to_mutate = choices(range(self.genome_size), k=self.genome_size)
        neighbor = list(individual)

        for index in index_to_mutate:
            neighbor[index] = 1 - neighbor[index]

        # Probability of accepting a worse solution
        temperature = max(1.0, self.iterations_without_improvement)
        acceptance_probability = self.acceptance_probability(self.best_fitness, self.problem(neighbor), temperature)

        if random() < acceptance_probability:
            return neighbor
        else:
            return individual

    @staticmethod
    def acceptance_probability(current_fitness, new_fitness, temperature):
        """
        Calculate the acceptance probability for Simulated Annealing.

        Args:
        - current_fitness: The fitness of the current solution.
        - new_fitness: The fitness of the new solution.
        - temperature: The current temperature.

        Returns:
        - probability: The acceptance probability.
        """
        if new_fitness > current_fitness:
            return 1.0
        else:
            return exp((new_fitness - current_fitness) / temperature)


# Usage of the Genetic Algorithm
problem_instances = [1, 2, 5, 10]

for instance in problem_instances:
    fitness_value = instance
    fitness = lab3_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value

    ga = GeneticAlgorithm(problem=fitness, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5)
    best_solution = ga.run()

    print(f"Best solution for problem instance {instance}: {''.join(map(str, best_solution))}")
    print(f"Best fitness: {fitness(best_solution):.2%}")
    print(f"Total fitness calls: {fitness.calls}\n")


Best solution for problem instance 1: 00000101101010010111011001101100100110000001110000000000110000110101100111001101110110010110101010001110111101100011111011011001101000110111001010011001111010011111110111011110100001100111110101001110001111110110101010010000001101111100101110111111111111010001010001111100101011101101100110010010000000111010111011000111110011111010101101011010111000101101000110111000101100111010111111111101000000000000100001011001100011100100101111000001001111000001001100110101011011000001011001100011010011110110100011110101100010010110010001110111011110011101110110111011110100110000111001001010001001011010111101001100101101101101110100001100011111010101111000101001111101110111111001010110101100111100101101111110001011010001111100111110010011010110100100010110011100110100111011001010111100111111111001100100101000101010010111111111001111100101010111110001111110000101100001101110000010011100100100001001111000001001011100010001011001001100000100101100101110101100110111

This code implements a Genetic Algorithm (GA) with Local Search based on Simulated Annealing. Compared to the previous version, it introduces convergence management with the `convergence_threshold` parameter. The algorithm terminates if the fitness does not improve for a certain consecutive number of generations.

Additionally, a tournament selection mechanism for choosing parents during crossover is incorporated with a size of 3 (`tournament_size=3`). This implies that 3 individuals are randomly chosen, and the best one is selected as a parent.

These modifications aim to enhance the exploration of the search space and introduce a mechanism to prevent prolonged executions without significant progress.

In [35]:
from random import choices, randint, random, uniform
from math import exp

class GeneticAlgorithm:
    def __init__(self, problem, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=50, convergence_threshold=5):
        # Initialize Genetic Algorithm parameters
        self.problem = problem
        self.genome_size = genome_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold  # New attribute

    def run(self):
        # Initialize the population
        population = [choices([0, 1], k=self.genome_size) for _ in range(self.population_size)]

        best_individual_after_local = None
        consecutive_no_improvement = 0  # Counter for consecutive generations without improvement

        for generation in range(self.generations):
            # Evaluate the fitness of the population
            fitness_scores = [self.problem(individual) for individual in population]
            best_individual = population[fitness_scores.index(max(fitness_scores))]

            # Apply local search to the best individual
            local_search = LocalSearch(self.problem, best_individual, iterations=self.local_search_iterations)
            local_search.run()
            best_individual_after_local = local_search.best_individual
            fitness_after_local = local_search.best_fitness

            # If fitness after local search improves, reset the counter
            if fitness_after_local > self.problem(best_individual):
                consecutive_no_improvement = 0
            else:
                consecutive_no_improvement += 1

            # If no improvement for a certain number of consecutive generations, terminate the algorithm
            if consecutive_no_improvement >= self.convergence_threshold:
                break

            # If fitness after local search improves, replace the individual in the population
            if fitness_after_local > self.problem(best_individual):
                index_to_replace = fitness_scores.index(min(fitness_scores))
                population[index_to_replace] = best_individual_after_local

            # Update the population using crossover and mutation
            new_population = self.crossover(population)
            self.mutate(new_population)
            population = new_population

        # Return the best individual after all generations
        return best_individual_after_local

    def crossover(self, population):
        # Use two-point crossover to generate new individuals from the existing population
        new_population = []

        for _ in range(self.population_size // 2):
            # Select parents through tournament selection
            parent1 = self.tournament_selection(population, tournament_size=3)
            parent2 = self.tournament_selection(population, tournament_size=3)

            crossover_points = sorted([randint(0, self.genome_size - 1) for _ in range(2)])

            child1 = parent1[:crossover_points[0]] + parent2[crossover_points[0]:crossover_points[1]] + parent1[crossover_points[1]:]
            child2 = parent2[:crossover_points[0]] + parent1[crossover_points[0]:crossover_points[1]] + parent2[crossover_points[1]:]

            new_population.extend([child1, child2])

        return new_population

    def mutate(self, population):
        # Apply mutation to each individual in the population with a certain probability
        for i in range(len(population)):
            for j in range(self.genome_size):
                if random() < self.mutation_rate:
                    population[i][j] = 1 - population[i][j]

    def tournament_selection(self, population, tournament_size):
        # Tournament selection: randomly choose participants and return the best one
        participants = choices(population, k=tournament_size)
        return max(participants, key=self.problem)


class LocalSearch:
    def __init__(self, problem, initial_solution, iterations=50):
        # Initialize Local Search parameters
        self.problem = problem
        self.genome_size = len(initial_solution)
        self.best_individual = initial_solution
        self.best_fitness = problem(self.best_individual)
        self.iterations_without_improvement = 0
        self.iterations = iterations

    def run(self):
        for _ in range(self.iterations):
            neighbor = self.generate_neighbor(self.best_individual)
            neighbor_fitness = self.problem(neighbor)

            if neighbor_fitness > self.best_fitness:
                self.best_individual = neighbor
                self.best_fitness = neighbor_fitness
                self.iterations_without_improvement = 0
            else:
                self.iterations_without_improvement += 1

            if self.best_fitness == 1.0:
                break  # Exit if the maximum fitness is reached

    def generate_neighbor(self, individual):
        # Simulated Annealing: Generate a neighbor by randomly flipping a bit with decreasing probability
        index_to_mutate = choices(range(self.genome_size), k=self.genome_size)
        neighbor = list(individual)

        for index in index_to_mutate:
            neighbor[index] = 1 - neighbor[index]

        # Probability of accepting the worse solution
        temperature = max(1.0, self.iterations_without_improvement)
        acceptance_probability = self.acceptance_probability(self.best_fitness, self.problem(neighbor), temperature)

        if random() < acceptance_probability:
            return neighbor
        else:
            return individual

    @staticmethod
    def acceptance_probability(current_fitness, new_fitness, temperature):
        if new_fitness > current_fitness:
            return 1.0
        else:
            return exp((new_fitness - current_fitness) / temperature)


# Usage of the genetic algorithm
problem_instances = [1, 2, 5, 10]

for instance in problem_instances:
    fitness_value = instance
    fitness = lab3_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value  

    ga = GeneticAlgorithm(problem=fitness, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=50, convergence_threshold=5)
    best_solution = ga.run()

    print(f"Best solution for problem instance {instance}: {''.join(map(str, best_solution))}")
    print(f"Best fitness: {fitness(best_solution):.2%}")
    print(f"Total fitness calls: {fitness.calls}\n")


Best solution for problem instance 1: 10110101111101010001011001101011001000000111111100010001010000100101100110010111000010011001010010011010001111110101001101000111110100011001110000111111010111000111000001101100100100111111100110011100001001011001111110100010011101011100010001110111110111110011010101000101011111011111101001000101010100101110110111111011101000101101111110111100000110110001010010000100110011100000111011110010101101111100011010101100110111111011001000111101001010111101111011101010111101100001010011111010010100010010111101101100000111010011101001000011101011111100100010111000101111010001101100101010001100011011101110000101111010011111011010001011110100111110010011111111111101111100111011110110111110100111111100000011101010001100111100100010010111111010101110111101000010011000011011010011000110101111011001010111001011101100110011111100110101111100100111111100100011101001100001110111110011101111110011111001011001110000000111010011001000110100010111110110001000010110100100

The algorithm aims to optimize the fitness of a population of binary genomes through generations of evolution. It employs local search to enhance individual solutions and monitors convergence to terminate promptly when there is no improvement.

In [36]:
import random

class GeneticAlgorithm:
    def __init__(self, problem, genome_size, population_size, generations, mutation_rate, local_search_iterations, convergence_threshold):
        # Initialize Genetic Algorithm parameters
        self.problem = problem
        self.genome_size = genome_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold

    def initialize_population(self):
        # Generate an initial random population
        return [random.choice([0, 1]) for _ in range(self.genome_size)]

    def mutate(self, genome):
        # Apply mutation to the genome
        mutated_genome = genome.copy()
        for i in range(self.genome_size):
            if random.random() < self.mutation_rate:
                mutated_genome[i] = 1 - mutated_genome[i]
        return mutated_genome

    def local_search(self, genome):
        # Perform local search using mutation
        best_genome = genome
        best_fitness = self.problem(best_genome)
        for _ in range(self.local_search_iterations):
            mutated_genome = self.mutate(genome)
            mutated_fitness = self.problem(mutated_genome)
            if mutated_fitness > best_fitness:
                best_genome = mutated_genome
                best_fitness = mutated_fitness
        return best_genome

    def run(self):
        # Initialize the population
        population = [self.initialize_population() for _ in range(self.population_size)]
        best_solution = None
        best_fitness = 0
        prev_best_fitness = 0
        convergence_count = 0

        for generation in range(self.generations):
            # Perform local search on each individual in the population
            for i in range(self.population_size):
                population[i] = self.local_search(population[i])
                fitness = self.problem(population[i])
                if fitness > best_fitness:
                    best_solution = population[i]
                    best_fitness = fitness

            # Check for convergence
            if generation > 0 and best_fitness == prev_best_fitness:
                convergence_count += 1
            else:
                convergence_count = 0

            if convergence_count >= self.convergence_threshold:
                break

            prev_best_fitness = best_fitness

        return best_solution

# Usage of the genetic algorithm
problem_instances = [1, 2, 5, 10]

for instance in problem_instances:
    fitness_value = instance
    fitness = lab3_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value  

    ga = GeneticAlgorithm(problem=fitness, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5)
    best_solution = ga.run()

    print(f"Best solution for problem instance {instance}: {''.join(map(str, best_solution))}")
    print(f"Best fitness: {fitness(best_solution):.2%}")
    print(f"Total fitness calls: {fitness.calls}\n")


Best solution for problem instance 1: 01101000111111101110101110111101010111000110100010110010001111100010010011101000011111111110101111011100111011011001111010100010111100011011101001001101011001000000110001100101111111110100001001110001111101001100011001000110110111101011011011011101100110011100101111010010101011110010010011011111111101010110101111111111110111001010111001101010000011001100110101110100101011001111111101101000101101111100110011101111100101101110101000101101101100111011101110100111100100111011001001101000101111000111011100111101110011110111101011111000111001110011011100100111111110000001010011101000111110010110100010111010101100011100010001011111010100110110000111111100110111111111111111111101100101100111000110111111011111011110011101111000111111111011011000111101001100000011011101100000010000110010011111111001001100111101010000111100010111111111001000101110101111110010100110110011111100101111101110111110101110100011111101010111111000010000011111101111111011111011111101

This code is an extension of the previous Genetic Algorithm implementation, introducing the concept of block mutation. Instead of flipping individual bits during mutation, it now flips entire blocks of bits. The size of these blocks is defined by the `block_size` parameter. The rest of the algorithm remains similar, including the initialization of the population, local search, convergence tracking, and early termination. The `block_size` modification provides a different way for genetic diversity and exploration during mutation.

In [37]:
import random

class GeneticAlgorithm:
    def __init__(self, problem, genome_size, block_size, population_size, generations, mutation_rate, local_search_iterations, convergence_threshold):
        # Initialization of Genetic Algorithm parameters
        self.problem = problem
        self.genome_size = genome_size
        self.block_size = block_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold

    def initialize_population(self):
        # Initialize the population with random binary values
        return [random.choice([0, 1]) for _ in range(self.genome_size)]

    def mutate(self, genome):
        # Apply mutation to the genome by flipping blocks of bits with a certain probability
        mutated_genome = genome.copy()
        for i in range(0, self.genome_size, self.block_size):
            if random.random() < self.mutation_rate:
                mutated_genome[i:i+self.block_size] = [1 - bit for bit in mutated_genome[i:i+self.block_size]]
        return mutated_genome

    def local_search(self, genome):
        # Perform local search by iteratively applying mutation and selecting the best fitness
        best_genome = genome
        best_fitness = self.problem(best_genome)
        for _ in range(self.local_search_iterations):
            mutated_genome = self.mutate(genome)
            mutated_fitness = self.problem(mutated_genome)
            if mutated_fitness > best_fitness:
                best_genome = mutated_genome
                best_fitness = mutated_fitness
        return best_genome

    def run(self):
        # Initialize the population and variables to track the best solution and fitness
        population = [self.initialize_population() for _ in range(self.population_size)]
        best_solution = None
        best_fitness = 0

        for generation in range(self.generations):
            # Apply local search to each individual in the population
            for i in range(self.population_size):
                population[i] = self.local_search(population[i])
                fitness = self.problem(population[i])
                if fitness > best_fitness:
                    best_solution = population[i]
                    best_fitness = fitness

            # Check for convergence
            if generation > 0 and best_fitness == prev_best_fitness:
                convergence_count += 1
            else:
                convergence_count = 0

            if convergence_count >= self.convergence_threshold:
                break

            prev_best_fitness = best_fitness

        return best_solution

# Usage of the genetic algorithm
problem_instances = [1, 2, 5, 10]

for instance in problem_instances:
    fitness_value = instance
    fitness = lab3_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value  

    # Create and run the genetic algorithm
    ga = GeneticAlgorithm(problem=fitness, genome_size=1000, block_size=20, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5)
    best_solution = ga.run()

    # Print the results
    print(f"Best solution for problem instance {instance}: {''.join(map(str, best_solution))}")
    print(f"Best fitness: {fitness(best_solution):.2%}")
    print(f"Total fitness calls: {fitness.calls}\n")


Best solution for problem instance 1: 01100001111001001111111101010001101100100111110100100100111110010011101101100110000111111110101100010001101110001111010111110100101011110011010100010110000110011111010110110100100110100110111010000111110111100110001111101111110100000111111111001101001000001010110111001110111000101101011100100101001001100001011101111100101110101111011000101100110111011001100000111101110111011010001111100111100010100111111011101011101110011011111111101011110101101111011110011110111110010101111010110101000101011110011101111011001101101110011111011101111011100001011100110111010010101111100101101110111001111110010011011110011011110000111000011001010101111011111111011110011111110110001101011000101011101110110001010001010011110011110110111001000101100111011010111001111101100001000000111111111001001001000100101100000011111000110011110100010100110101101100111111101111101110011101111110001001101101011111001100010010111111110101010001110000111110001000000001111010111101100011